In [12]:
with open("data/bert/data.txt", "r", encoding="utf-8") as f:
    data = f.read().split("\n")

In [13]:
len(data)

7170932

In [14]:
from tokenizers import BertWordPieceTokenizer
from nltk import sent_tokenize

In [15]:
tokenizer = BertWordPieceTokenizer("assets/kor-vocab.txt", lowercase=False)

In [16]:
new_data = []
num_tokens = 0

In [17]:
min_token_length = 5
max_token_length = 510

In [18]:
import re

def clean_text(text):    
    text = re.sub(r"\s+", " ", text)
    text = text.replace(". ", ".")
    text = re.sub(r"([ㄱ-ㅎㅏ-ㅣ가-힣])\.([\S])", r"\1. \2", text)
    text = re.sub(r"([\S])\.([ㄱ-ㅎㅏ-ㅣ가-힣])", r"\1. \2", text)
    text = text.strip()
    
    return text

In [19]:
import time
from tqdm import tqdm

start = time.time()
for row in tqdm(data, total=len(data)):
    tokens = []
    if len(tokenizer.encode(row, add_special_tokens=False).ids) <= min_token_length:
        continue
    sentences = sent_tokenize(row)
    for sent in sentences:
        tokens_ = tokenizer.encode(sent, add_special_tokens=False).ids
        if len(tokens) + len(tokens_) <= max_token_length:
            tokens += tokens_
        else:
            new_data.append(clean_text(tokenizer.decode(tokens)))
            num_tokens += len(tokens)
            tokens = tokens_[:max_token_length]
    new_data.append(clean_text(tokenizer.decode(tokens)))
    num_tokens += len(tokens)
    tokens = []
with open("data/bert/data_processed.txt", "w", encoding="utf-8") as f:
    for row in new_data:
        f.write(row+"\n")
end = time.time()
print(end-start)

100%|██████████| 7170932/7170932 [1:41:01<00:00, 1183.07it/s]


6079.199527025223


In [20]:
num_tokens

432347819

In [21]:
len(data)

7170932

In [22]:
len(new_data)

6160035

In [23]:
import random
random.seed(1)
random.shuffle(new_data)

In [24]:
with open("data/bert/train_data.txt", "w") as f:
    for row in new_data[:-100000]:
        f.write(row+"\n")
    f.close()
with open("data/bert/dev_data.txt", "w") as f:
    for row in new_data[-100000:]:
        f.write(row+"\n")
    f.close()